# Adding Post-Newtonian general relativity corrections

It's easy to add post-newtonian corrections to your REBOUND simulations with REBOUNDx.  Let's start with a simulation without GR:

In [1]:
import rebound
sim = rebound.Simulation()
sim.add(m=1.) # Sun
sim.add(m=1.66013e-07,a=0.387098,e=0.205630) # Mercury-like
sim.move_to_com() # Moves to the center of momentum frame

sim.integrate(10.)
print("pomega = %.16f"%sim.particles[1].pomega)

pomega = 0.0000000000000000


As expected, the pericenter did not move at all.  Now let's add GR

In [2]:
import reboundx
rebx = reboundx.Extras(sim)
params = rebx.add_gr()
E0 = rebx.gr_hamiltonian(sim, params)

and run for another 100 time units

In [3]:
deltat = 100.
sim.integrate(sim.t + deltat)
print("pomega = %.16f"%sim.particles[1].pomega)
juliancentury = 628.33195 # in yr/2pi
arcsec = 4.8481368e-06 # in rad
print("Rate of change of pomega = %.4f [arcsec / Julian century]"% (sim.particles[1].pomega/deltat*juliancentury/arcsec))
Ef = rebx.gr_hamiltonian(sim, params)
print("Relative error on the relativistic Hamiltonian = {0}".format(abs(Ef-E0)/abs(E0)))

pomega = 0.0000332618418421
Rate of change of pomega = 43.1083 [arcsec / Julian century]
Relative error on the relativistic Hamiltonian = 1.234409293531869e-16


As expected there was pericenter precession. The literature value is 42.98 arcsec / century.   

**Units**

To add GR, you need to pass `add_gr` the speed of light `c` in the units appropriate to the simulation.  By not passing a value above, `c` defaulted to our default units of AU, $M_\odot$ and yr/$2\pi$, for which `G=1` which is what we used for our initial conditions above.

But imagine now we wanted to instead use SI units:

In [4]:
sim = rebound.Simulation()
sim.G = 6.67408e-11
sim.add(m=1.989e30) # Sun
sim.add(m=3.30216458e23,a=5.79090366e10,e=0.205630) # SI units
sim.move_to_com() # Moves to the center of momentum frame
rebx = reboundx.Extras(sim)

We now have to explicitly pass `c`:

In [5]:
params = rebx.add_gr(c=3.e8)

Currently, setting the `units` member in Simulation does not work with REBOUNDx.  If you want to use your own units you always have to pass `c` explicitly.

**Variants**

Above we always called `add_gr`, but there are two other options `add_gr_potential` and `add_gr_full`.  Before running any serious simulations, you should read the more detailed descriptions at http://reboundx.readthedocs.io/en/latest/effects.html to see which implementation is appropriate for your application.

**Source**

For the implementations that treat a single body as massive (`add_gr` and `add_gr_potential`), you can also specify which particle in the simulation is the massive body responsible for the post-newtonian corrections by passing the index of the body in the `particles` array, e.g.

In [6]:
params = rebx.add_gr_potential(source_index = 0, c=3.e8)

If you don't pass a source_index, it will default to `particles[0]`, so in this case we didn't have to pass it.